# Cómo lidiar con datos faltantes

Vamos a importar el siguiente dataset "empleados.csv"


In [1]:
# Importar librerias
import pandas as pd
import numpy as np

# Leer el csv como un pandas dataFrame
df = pd.read_csv("empleados.csv")

# Imprimir las primeras filas
print(df.head())
df

  First Name  Gender  Salary Bonus % Senior Management             Team
0    Douglas    Male   97308   6.945              TRUE        Marketing
1     Thomas    Male   61933     NaN              TRUE              NaN
2      Maria  Female  130590  11.858             FALSE          Finance
3      Jerry    Male     NaN    9.34              TRUE          Finance
4      Larry    Male  101004   1.389              TRUE  Client Services


,First Name,Gender,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,97308,6.945,TRUE,Marketing
1,Thomas,Male,61933,NaN,TRUE,NaN
2,Maria,Female,130590,11.858,FALSE,Finance
3,Jerry,Male,NaN,9.34,TRUE,Finance
4,Larry,Male,101004,1.389,TRUE,Client Services
...,...,...,...,...,...,...
995,Henry,NaN,132483,16.655,FALSE,Distribution
996,Phillip,Male,42392,19.675,FALSE,Finance
997,Russell,Male,96914,1.421,FALSE,Product
998,Larry,Male,60500,11.985,FALSE,Business Development


Tiene 1000 columnas con 8 variables. Se pueden obtener estadísitcas básicas con los métodos `.dtypes` y `.describe()`.

In [ ]:
print(df.dtypes)


In [ ]:
print(df.describe())

Note que los tipos de datos de todas la columnas es "object". Pero esto no debería se el caso para  Salary, Senior Management y Bonus. Esto sucede porque tenemos **valores corruptos en estas columnas**. Una vez que resolvamos los valores faltantes, podremos convertir las columnas en los tipos de datos requeridos con el método `.astype()`. 

## Como marcar valores inválidos o corruptos como datos faltantes

Pandas trata None y NaN como esencialmente intercambiables para indicar valores faltantes o nulos. Otros valores como na y ? Pandas no los reconoce de forma predeterminada. Centrémonos en la columna de Salary.
 

In [2]:
print('Salary')
print(df['Salary'].head(10))

Salary
0     97308
1     61933
2    130590
3       NaN
4    101004
5    115163
6     65476
7     45906
8       NaN
9    139852
Name: Salary, dtype: object


En la 8va fila hay un valor faltanta y en la 3ra fila hay un NA, los cuales Pandas automáticamente llena con "NaN". Pero que pasa con otros símbolos como ?, n.a., etc. Veamos la columna "Gender".

In [4]:
print(df['Gender'].head(10))

0      Male
1      Male
2    Female
3      Male
4      Male
5      n.a.
6    Female
7    Female
8       NaN
9    Female
Name: Gender, dtype: object


In [3]:
df['Gender'].unique()

array(['Male', 'Female', 'n.a.', nan], dtype=object)

Notamos que n.a. no se ha convertido en NaN y mantiene su forma original. 
Podemos entregar estos formatos en el método `.read_csv()` para permitir que Pandas los reconozca como datos corruptos o faltantes. 

In [5]:
# una lista con todos los formatos asociados a valores faltantes
missing_value_formats = ["n.a.","?","NA","n/a", "na", "--","..."]
df = pd.read_csv("empleados.csv", na_values = missing_value_formats)

#print gender denuevo
print(df['Gender'].head(10))

0      Male
1      Male
2    Female
3      Male
4      Male
5       NaN
6    Female
7    Female
8       NaN
9    Female
Name: Gender, dtype: object


In [6]:
df['Gender'].unique()

array(['Male', 'Female', nan], dtype=object)

Hasta ahora, nuestros valores faltantes tenían identificadores únicos que los hacían bastante fáciles de detectar. Pero, ¿qué sucede cuando obtenemos un tipo de datos no válido? He diseñado una función que me permite buscar tipos de datos no válidos en una columna.

In [7]:
import pandas as pd

missing_value_formats = ["n.a.","?","NA","n/a", "na", "--","999"]
df = pd.read_csv("empleados.csv", na_values = missing_value_formats)

def make_int(i):
    try:
        return int(i)
    except:
        return pd.np.nan

# aplicar la función make_int  a las series usando map
df['Salary'] = df['Salary'].map(make_int)
print(df['Salary'].head())


0     97308.0
1     61933.0
2    130590.0
3         NaN
4    101004.0
Name: Salary, dtype: float64


<ipython-input-7-fda9efc20ff5>:10: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  return pd.np.nan


### Marcando valores perdidos usando isnull y notnull

En Pandas, tenemos dos funciones:   
- `isnull()` función para marcar todos los valores NaN en el dataset como True  
- `notnull()` para marcar todos los valores NaN en el dataset como False.


In [8]:
print(df['Gender'].isnull().head(10)) # NaN values are marked True
print(df['Gender'].notnull().head(10)) # non-NaN values are marked True


0    False
1    False
2    False
3    False
4    False
5     True
6    False
7    False
8     True
9    False
Name: Gender, dtype: bool
0     True
1     True
2     True
3     True
4     True
5    False
6     True
7     True
8    False
9     True
Name: Gender, dtype: bool


Podemos usar los resultados de las funciones `isnull` y `notnull` para filtrar. Imprimamos todas las filas de la database para las cuales Gender no es faltante. 

In [ ]:
# Retorna True en los índices para los cuales Gender no es NaN
null_filter = df['Gender'].notnull()
print(df[null_filter].head(10))

### Estadísticas de valores perdidos
`isnull` y `notnull` también se puede utilizar para resumir los valores perdidos. 

Para verificar si existe algún valor faltante en nuestro dataframe:

In [9]:
print(df.isnull().values.any())

True


Número total de valores perdidos por columna:

In [10]:
print(df.isnull().sum())

First Name            70
Gender               149
Salary                 5
Bonus %                4
Senior Management     71
Team                  48
dtype: int64


### Cómo remover filas con valores faltantes

La librería de Pandas provee la función `dropna()` que puede ser usada para remover filas y columnas con datos faltantes.

En el ejemplo a continuación, usamos dropna() para remover todas las filas con datos faltantes:

In [11]:
# remover todas las filas con valores NaN
new_df = df.dropna(axis=0)

# Validar si existe algún valor NaN en nuestro dataset
print(new_df.isnull().values.any())


False


Podemos también usar el parámetro `how`.
- `how = 'any'`: al menos un valor es nulo.
- `how = 'all'`: todos los valores deben ser nulos.

In [ ]:
# remover todas las filas con al menos un NaN
new_df = df.dropna(axis = 0, how ='any')  

# remover todas las filas con todos los valores NaN
new_df = df.dropna(axis = 0, how ='all')

# remover todas las columnas con al menos un NaN
new_df = df.dropna(axis = 1, how ='any')

# remover todas las columnas con todos los valores NaN
new_df = df.dropna(axis = 1, how ='all')


In [13]:
new_df = df.dropna(axis = 0, how ='all')
new_df
df

,First Name,Gender,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,97308.0,6.945,True,Marketing
1,Thomas,Male,61933.0,NaN,True,NaN
2,Maria,Female,130590.0,11.858,False,Finance
3,Jerry,Male,NaN,9.340,True,Finance
4,Larry,Male,101004.0,1.389,True,Client Services
...,...,...,...,...,...,...
995,Henry,NaN,132483.0,16.655,False,Distribution
996,Phillip,Male,42392.0,19.675,False,Finance
997,Russell,Male,96914.0,1.421,False,Product
998,Larry,Male,60500.0,11.985,False,Business Development


In [12]:
new_df = df.dropna(axis = 0, how="any", thresh=6)
new_df

,First Name,Gender,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,97308.0,6.945,True,Marketing
2,Maria,Female,130590.0,11.858,False,Finance
4,Larry,Male,101004.0,1.389,True,Client Services
6,Ruby,Female,65476.0,10.012,True,Product
9,Frances,Female,139852.0,7.524,True,Business Development
...,...,...,...,...,...,...
994,George,Male,98874.0,4.479,True,Marketing
996,Phillip,Male,42392.0,19.675,False,Finance
997,Russell,Male,96914.0,1.421,False,Product
998,Larry,Male,60500.0,11.985,False,Business Development


### Reemplazando NaNs con un simple valor constante

Vamos a usar `fillna()` para reemplazar valores en la columna **Salary** con 0.


In [ ]:
df['Salary'].fillna(0)

También podemos ocupar la misma función para variables categóricas como **Gender**.

In [ ]:
df['Gender'].fillna('No Gender')

### Reemplazando NaNs con los valores de la fila anterior

Es una solución común cuando se llena valores faltantes en datos de imágenes. se usa `method = 'pad'`. Probemos para la columna Salary:

In [ ]:
df['Salary'].fillna(method='pad')

### Reemplazando NaNs con el valor de la siguiente fila
Usamos `method = 'bfill'`.

In [ ]:
df['Salary'].fillna(method='bfill')

### Reemplazamos NaNs usando Median/Mean de la columnas

In [ ]:
# usando median
df['Salary'].fillna(df['Salary'].median())

In [ ]:
#usando mean
df['Salary'].fillna(int(df['Salary'].mean()))

### Usando un método de reemplazo
Usar un método de reemplazo es la forma más generica de resolver el problema fillna method. A continuación, especificamos tanto el valor a reemplazar como el valor de reemplazo.

In [ ]:
# reemplazaremos el valor NaN en Salary con el valor 0  

df['Género']=df['Gender'].replace(to_replace = "Male", value = "Varones").replace(to_replace = "Female", value = "Mujer")
df

### Usando el método de interpolación
La función `interpolate()` es usada para llenar los valores NaN  usando varias tecnicas de interpolación.
Veamos la interpolación de datos faltantes usando el método de Interpolación Lineal

In [14]:
df['Salary'].interpolate(method='linear', direction = 'forward')

0       97308.0
1       61933.0
2      130590.0
3      115797.0
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64

In [15]:

df['Salary']

0       97308.0
1       61933.0
2      130590.0
3           NaN
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64